In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.decomposition import PCA
import pylab as pl
from sklearn import preprocessing
from sklearn import ensemble

In [2]:
data = pd.read_csv("data.csv")
df = pd.DataFrame(data) # Converting data to Panda DataFrame
df.describe() #gives statistics about the columns of the dataframe

,Input,instructionCount,ilp32,ilp256,ilp2048,ilp65536,memops,ctrlops,intops,flops,...,lbdiv16,lbdiv32,lbdiv64,lbdiv128,lbdiv256,lbdiv512,lbdiv1024,probsize,CPU Time,GPU Time
count,8.300000e+01,8.300000e+01,83.000000,83.000000,83.000000,83.000000,83.000000,83.000000,83.000000,83.000000,...,8.300000e+01,8.300000e+01,8.300000e+01,8.300000e+01,8.300000e+01,8.300000e+01,8.300000e+01,8.300000e+01,83.000000,83.000000
mean,3.523174e+07,1.076400e+10,4.852934,10.607886,74.282014,563.371791,0.712525,0.120555,0.740644,0.178620,...,1.062817e-03,1.741444e-03,2.145406e-03,2.501448e-03,3.088642e-03,2.871509e+01,5.409485e-03,4.152963e+05,22.571006,22.765411
std,1.514525e+08,2.194167e+10,1.591059,8.206145,148.368694,1774.465515,0.105899,0.074426,0.113245,0.140009,...,3.909190e-03,6.358476e-03,7.161677e-03,7.357479e-03,7.849943e-03,2.615678e+02,1.167974e-02,1.335055e+06,34.422967,63.084143
min,2.560000e+02,2.611762e+06,1.442210,1.918572,1.980879,2.570037,0.461629,0.007327,0.518805,0.000000,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,2.000000e+00,0.016934,0.124418
25%,1.368000e+03,4.881504e+08,4.000063,4.458856,5.007727,5.773378,0.626536,0.072202,0.620232,0.050000,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,9.000000e+00,2.773428,0.760976
50%,2.000000e+04,2.239865e+09,5.000074,9.059592,9.687678,14.166241,0.718750,0.100001,0.777779,0.185172,...,6.560000e-08,1.310000e-07,2.630000e-07,4.750000e-07,6.680000e-07,7.420000e-07,8.350000e-07,2.970000e+02,10.475384,3.324654
75%,1.733445e+05,8.364610e+09,6.141176,13.489561,33.253687,255.673730,0.808426,0.145731,0.831889,0.245617,...,1.185536e-04,1.929116e-04,2.952818e-04,3.926128e-04,3.956721e-04,5.597367e-04,6.199144e-04,2.469500e+03,28.668817,14.161183
max,1.000000e+09,1.374371e+11,7.197260,37.911765,534.000000,9023.000000,0.890129,0.307693,0.886846,0.523809,...,2.225168e-02,3.323731e-02,3.596328e-02,3.596328e-02,3.596328e-02,2.383000e+03,3.705571e-02,6.944444e+06,169.931417,412.397738


In [3]:
#seperate traindata and test data here

In [4]:
## Class Distribution

In [5]:
yes = len(data[data.Class == "Y"])
No = len(data[data.Class == "N"])

print yes
print No

66
17


In [6]:
## Normalizing data to reside in between 0 and 1 using MinMaxScaler

In [7]:
min_max_scaler = preprocessing.MinMaxScaler()

dfTest = min_max_scaler.fit_transform(df[['instructionCount','blocks','pages',
                                          'ilp32','ilp256','ilp2048','ilp65536','probsize']])

X_train = df.drop(['CPU Time','GPU Time','Class','dataSet','Input','instructionCount','ilp32','ilp32'
                ,'ilp256','ilp2048','ilp65536','pages','blocks','coldref','reuseDist2','probsize'],axis=1)

X_train['instructionCount'] = pd.DataFrame({'instructionCount':dfTest[:,0]})
X_train['blocks']           = pd.DataFrame({'blocks':dfTest[:,1]})
X_train['pages']            = pd.DataFrame({'pages':dfTest[:,2]})
X_train['ilp32']            = pd.DataFrame({'ilp32':dfTest[:,3]})
X_train['ilp256']           = pd.DataFrame({'ilp256':dfTest[:,4]})
X_train['ilp2048']          = pd.DataFrame({'ilp2048':dfTest[:,5]})
X_train['ilp65536']         = pd.DataFrame({'ilp65536':dfTest[:,6]})
X_train['probsize']         = pd.DataFrame({'probsize':dfTest[:,6]})

X_train.head()

,memops,ctrlops,intops,flops,coldref fraction,resuseDist2 fraction,sfp,dfp,noconflict,broadCast,...,lbdiv512,lbdiv1024,instructionCount,blocks,pages,ilp32,ilp256,ilp2048,ilp65536,probsize
0,0.820424,0.051388,0.846086,0.256314,0.000981,0.625033,0.400000,0.0,0.067171,0.899166,...,0.000000,0.000000,0.000172,3.075188e-07,0.000008,0.878528,0.308208,0.020860,0.001486,0.001486
1,0.820468,0.051335,0.846120,0.256362,0.000979,0.625014,0.400000,0.0,0.051082,0.924311,...,0.000000,0.000000,0.000743,3.082595e-07,0.000008,0.878686,0.308042,0.020786,0.001305,0.001305
2,0.820476,0.051325,0.846126,0.256371,0.000978,0.625011,0.400000,0.0,0.051449,0.928102,...,0.000000,0.000000,0.001116,3.083987e-07,0.000008,0.819275,0.308012,0.020772,0.001276,0.001276
3,0.666661,0.074120,0.666661,0.185172,0.000009,0.555550,0.599922,0.0,0.062615,0.937385,...,0.004560,0.009120,0.168962,3.181174e-05,0.000835,0.922278,0.197610,0.013819,0.001278,0.001278
4,0.642840,0.071516,0.678563,0.178545,0.000019,0.555543,0.599844,0.0,0.062729,0.937271,...,0.017669,0.035338,0.021787,1.583899e-05,0.000415,0.965575,0.207809,0.015128,0.001461,0.001461


In [8]:
Y_train = df['Class']

X_train = np.asarray(X_train)
Y_train = np.asarray(Y_train)

In [9]:
#test dataset

test = pd.read_csv("test.csv")
df_test = pd.DataFrame(test) #reading test dataset

min_max_scaler = preprocessing.MinMaxScaler()

dfTest_Test = min_max_scaler.fit_transform(df_test[['instructionCount','blocks','pages',
                                               'ilp32','ilp256','ilp2048','ilp65536','probsize']])

X_test = df_test.drop(['CPU Time','GPU Time','Class','dataSet','Input','instructionCount','ilp32','ilp32'
                   ,'ilp256','ilp2048','ilp65536','pages','blocks','coldref','reuseDist2','probsize'],axis=1)

X_test['instructionCount'] = pd.DataFrame({'instructionCount':dfTest_Test[:,0]})
X_test['blocks'] = pd.DataFrame({'blocks':dfTest_Test[:,1]})
X_test['pages'] = pd.DataFrame({'pages':dfTest_Test[:,2]})
X_test['ilp32'] = pd.DataFrame({'ilp32':dfTest_Test[:,3]})
X_test['ilp256'] = pd.DataFrame({'ilp256':dfTest_Test[:,4]})
X_test['ilp2048'] = pd.DataFrame({'ilp2048':dfTest_Test[:,5]})
X_test['ilp65536'] = pd.DataFrame({'ilp65536':dfTest_Test[:,6]})
X_test['probsize'] = pd.DataFrame({'probsize':dfTest_Test[:,6]})

Y_test =df_test['Class'] # We create our label

X_test = np.asarray(X_test)
Y_test = np.asarray(Y_test)


## Ensembled Bagging Decison Tree Classifier

In [10]:
baggingClassifier = ensemble.BaggingClassifier(n_estimators=11,bootstrap_features=True,n_jobs=-1)
baggingClassifier.fit(X_train,Y_train)

BaggingClassifier(base_estimator=None, bootstrap=True,
         bootstrap_features=True, max_features=1.0, max_samples=1.0,
         n_estimators=11, n_jobs=-1, oob_score=False, random_state=None,
         verbose=0, warm_start=False)

## cross validating the model

In [11]:
scores = cross_val_score(baggingClassifier, X_train, Y_train)
print scores

[0.78571429 0.82142857 0.55555556]


In [12]:
print baggingClassifier.predict(X_test)
print Y_test

['N' 'N' 'N' 'Y' 'Y' 'N']
['N' 'N' 'N' 'Y' 'Y' 'Y']
